In [1]:
import sys
import time 

import pg8000.native
import psycopg2
from faker import Faker

In [2]:
try:
    con = psycopg2.connect(database="postgres", user="postgres", password="1", host="127.0.0.1", port="5432")
    cur = con.cursor()
    print("Cool!", con.closed)
except BaseException as be:
    print("Impossible to connect to db:", be)
    sys.exit(1)

Cool! 0


In [11]:
con.commit()

In [37]:
con.reset()

In [7]:
# create tables
cur.execute("drop table if exists purchases;")
cur.execute("drop table if exists purchases_products_list;")
cur.execute("drop table if exists products;")
cur.execute("drop table if exists sales;")
cur.execute("drop table if exists customer;")
cur.execute("create table if not exists customer (id integer primary key, name text, address text, review text)")
cur.execute("create table if not exists purchases (id integer primary key, customer_id int, constraint a foreign key (customer_id) references customer(id)); ")
cur.execute("create table if not exists products (id integer primary key, name text, details text, price int, type text)")
cur.execute("create table if not exists purchases_products_list (id integer primary key, purchase_id int, product_id int, constraint b foreign key (purchase_id) references purchases(id), constraint c foreign key (product_id) references products(id))")
cur.execute("create table if not exists sales (id integer primary key, type text, discount int)")
con.commit()

In [86]:
#generating values
import random
fake = Faker()

try: 
    cur.execute("delete from purchases_products_list; \
    delete from purchases; \
    delete from products; \
    delete from sales; \
    delete from customer;")
    for i in range(101):
        cur.execute(f"insert into customer (id, name, address, review) values ('{str(i)}', '{fake.name()}', '{fake.address()}', '{fake.text()}');")
    for i in range(301):
        cur.execute(f"insert into purchases (id, customer_id) values ('{str(i)}', {random.randint(0, 100)});")
    types = fake.words(20)
    for i in range(301):
        cur.execute(f"insert into products (id, name, details, price, type) values ('{str(i)}', '{fake.text(25)}', '{fake.text(50)}', {random.randint(0, 1000)}, '{types[random.randint(0, len(types)-1)]}');")
    for i in range(3000):
        cur.execute(f"insert into purchases_products_list (id, purchase_id, product_id) values ('{str(i)}', {random.randint(0, 300)}, {random.randint(0, 300)});")
    for i in range(10):
        cur.execute(f"insert into sales (id, type, discount) values ({str(i)}, '{types[random.randint(0, len(types) // 3)]}', {random.randint(0,30)});")
    con.commit()
except BaseException as e: 
    print("Error!", e)
    con.reset()

In [104]:
con.reset()

In [107]:
# Select all products purchased by the customer with discount

cur.execute("select customer.name as Name, products.name as \"Product Name\", sales.type from sales join products on sales.type = products.type join purchases_products_list p on products.id = p.product_id join purchases on p.purchase_id = purchases.id join customer on  purchases.customer_id = customer.id \
    order by customer.name, sales.type")
for r in cur:
    print(r)

('Aaron Smith', 'Industry air sport but.', 'a')
('Aaron Smith', 'Walk thank community.', 'a')
('Aaron Smith', 'Walk thank community.', 'a')
('Aaron Smith', 'Chance far no.', 'a')
('Aaron Smith', 'Couple hope like.', 'a')
('Aaron Smith', 'Industry air sport but.', 'a')
('Aaron Smith', 'Couple hope like.', 'a')
('Aaron Smith', 'Chance far no.', 'a')
('Aaron Smith', 'Bill cost return.', 'by')
('Aaron Smith', 'Significant stand fact.', 'by')
('Aaron Smith', 'Fall phone world.', 'by')
('Aaron Smith', 'Staff one play for feel.', 'hair')
('Aaron Smith', 'Star begin guy leader.', 'hair')
('Aaron Smith', 'Star begin guy leader.', 'hair')
('Aaron Smith', 'Staff one play for feel.', 'hair')
('Aaron Smith', 'Boy field require.', 'rest')
('Aaron Smith', 'Food whom begin dream.', 'rest')
('Aaron Smith', 'Beyond since form old.', 'rest')
('Aaron Smith', 'Fine they word much.', 'turn')
('Aaron Smith', 'Fine they word much.', 'turn')
('Aaron Smith', 'Fine they word much.', 'turn')
('Aaron Smith', 'Size

In [108]:
# Find the total saved money for each customer

cur.execute("select customer.name, sum(products.price::int) - sum(products.price::int * (100 - sales.discount)/100) as \"Saved Money\" from sales join products on sales.type = products.type join purchases_products_list p on products.id = p.product_id join purchases on p.purchase_id = purchases.id join customer on  purchases.customer_id = customer.id \
    group by customer.name order by customer.name")
for r in cur:
    print(r)

('Aaron Smith', 2224)
('Allison Jordan', 1883)
('Amanda Rodriguez', 4)
('Amanda Wagner', 1727)
('Andrea Neal', 1854)
('Angela Black', 2003)
('Angela Brown', 541)
('Angela Robinson', 1262)
('Angela Walker', 534)
('Angela Wilson', 345)
('Anthony Ruiz', 906)
('Barry Howell', 508)
('Brian Austin', 764)
('Bryan Williams', 1850)
('Carrie Bennett', 627)
('Catherine Mcdonald', 4807)
('Charles Wilkins', 827)
('Chase Shah', 2351)
('Christopher Mcfarland', 1017)
('Colin Robertson', 145)
('Danielle Richardson', 1054)
('Danny Terry', 1109)
('David Barker', 2422)
('David Koch', 189)
('David Love', 843)
('Deborah Jones', 1133)
('Denise Gardner', 895)
('Diana Wheeler', 769)
('Elizabeth Charles', 322)
('Eric Edwards', 338)
('Erica Howard', 1852)
('Gabriela Reid', 878)
('Harry Johnson', 551)
('Jamie Barker', 918)
('Jennifer Caldwell', 389)
('Jennifer Rodriguez', 636)
('Joshua Conway', 1697)
('Joshua Horn', 308)
('Justin Gray', 1678)
('Justin Hale', 325)
('Kathleen Adams', 1193)
('Katie Boyd', 481)
('Kat

In [105]:
cur.execute("create index a on products(type);")
cur.execute("create index b on sales(type);")